In [2]:
import numpy as np
from dualtime.src.mclachlan import VarQITE
from gibbs.utils import *
from gibbs.preparation.varqite import efficient_su2_ansatz,brute_force_optimization
from qiskit.opflow import PauliSumOp
from qiskit.quantum_info import SparsePauliOp
%load_ext autoreload
%autoreload 2

ModuleNotFoundError: No module named 'gibbs.preparation.utils'

In [9]:

beta = 1.0
hamiltonian = create_hamiltonian_lattice(3,1.3,1.7)
hamiltonian = SparsePauliOp.from_list([("XY",1)])
N = hamiltonian.num_qubits
entanglement = [[i,N+i] for i in range(N)] + [[N+i,i+1] for i in range(N-1)]
entanglement = entanglement[::2] + entanglement[1::2]
ansatz,x0 = efficient_su2_ansatz(hamiltonian,depth = 1,entanglement="full")
display(ansatz.decompose().draw(output='text'))
varqite = VarQITE(ansatz,x0,backend=None)
result = varqite.evolve(PauliSumOp(hamiltonian^("I"*N)),beta/2,timestep = 0.01)


brute_params = brute_force_optimization(hamiltonian,ansatz,x0,beta = beta).x


┌──────────┐┌──────────┐ ░                                ░  ┌──────────┐»
q_0: ┤ Ry(θ[0]) ├┤ Rz(θ[4]) ├─░───■────■────■──────────────────░──┤ Ry(θ[8]) ├»
     ├──────────┤├──────────┤ ░ ┌─┴─┐  │    │                  ░  ├──────────┤»
q_1: ┤ Ry(θ[1]) ├┤ Rz(θ[5]) ├─░─┤ X ├──┼────┼────■────■────────░──┤ Ry(θ[9]) ├»
     ├──────────┤├──────────┤ ░ └───┘┌─┴─┐  │  ┌─┴─┐  │        ░ ┌┴──────────┤»
a_0: ┤ Ry(θ[2]) ├┤ Rz(θ[6]) ├─░──────┤ X ├──┼──┤ X ├──┼────■───░─┤ Ry(θ[10]) ├»
     ├──────────┤├──────────┤ ░      └───┘┌─┴─┐└───┘┌─┴─┐┌─┴─┐ ░ ├───────────┤»
a_1: ┤ Ry(θ[3]) ├┤ Rz(θ[7]) ├─░───────────┤ X ├─────┤ X ├┤ X ├─░─┤ Ry(θ[11]) ├»
     └──────────┘└──────────┘ ░           └───┘     └───┘└───┘ ░ └───────────┘»
«     ┌───────────┐          
«q_0: ┤ Rz(θ[12]) ├──■───────
«     ├───────────┤  │       
«q_1: ┤ Rz(θ[13]) ├──┼────■──
«     ├───────────┤┌─┴─┐  │  
«a_0: ┤ Rz(θ[14]) ├┤ X ├──┼──
«     ├───────────┤└───┘┌─┴─┐
«a_1: ┤ Rz(θ[15]) ├─────┤ X ├
«     └───────────┘     └───┘

(-1.3862943611198908+0j)
(-0.970116539379804+0j)
(-0.9701165393798041+0j)
(-0.970116539379804+0j)
(-0.9701165393798038+0j)
(-1.3862943611198904+0j)
(-1.3862943611198904+0j)
(-1.3862943611198904+0j)
(-1.3862943611198904+0j)
(-0.9701165393798041+0j)
(-0.9701165393798041+0j)
(-0.9701165393798041+0j)
(-0.9701165393798039+0j)
(-1.3862943611198904+0j)
(-1.3862943611198908+0j)
(-1.3862943611198908+0j)
(-1.3862943611198908+0j)
(-1.0129803088591034+0j)
(-1.2498849705349548+0j)
(-1.354728564903749+0j)
(-1.3785123363154346+0j)
(-1.3532427576081227+0j)
(-1.3862943611198908+0j)
(-1.3647264384138968+0j)
(-1.3862943611198906+0j)
(-1.355547535241672+0j)
(-1.3862943611198908+0j)
(-1.3556821503855412+0j)
(-1.3862943611198908+0j)
(-1.3559277349187204+0j)
(-1.3862943611198908+0j)
(-1.3519828587838707+0j)
(-1.3862943611198908+0j)
(-1.3527546428857402+0j)
(-1.3862943611198908+0j)
(-1.3527546428857402+0j)
(-1.3862943611198908+0j)
(-1.3527546428857402+0j)
(-1.3776940801162865+0j)
(-1.3842615377830476+0j)
(-1.

In [10]:

# print(result.times)
# print(result.energies)

final_state = state_from_ansatz(ansatz,result.parameters[-1])

brute_state= state_from_ansatz(ansatz,brute_params)

print("The max difference is:", np.max(np.abs(final_state-expected_state(hamiltonian,beta))))
printarray(final_state,3,np.abs)
printarray(expected_state(hamiltonian,beta),3,np.abs)
printarray(brute_state,3,np.abs)

The max difference is: 0.19039853898894127
[[0.25 0.   0.   0.  ]
 [0.   0.25 0.   0.  ]
 [0.   0.   0.25 0.  ]
 [0.   0.   0.   0.25]]
[[0.25 0.   0.   0.19]
 [0.   0.25 0.19 0.  ]
 [0.   0.19 0.25 0.  ]
 [0.19 0.   0.   0.25]]
[[0.25 0.   0.   0.  ]
 [0.   0.25 0.   0.  ]
 [0.   0.   0.25 0.  ]
 [0.   0.   0.   0.25]]


In [4]:
from surfer.gradient import ReverseGradient
gradient = ReverseGradient(do_checks=False)
print(gradient.compute(hamiltonian^("I"*N), ansatz, x0))
display(ansatz.decompose().draw())

[ 1.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
 -2.22044605e-16 -2.22044605e-16  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]


┌──────────┐┌──────────┐ ░                                ░  ┌──────────┐»
q_0: ┤ Ry(θ[0]) ├┤ Rz(θ[4]) ├─░───■────■────■──────────────────░──┤ Ry(θ[8]) ├»
     ├──────────┤├──────────┤ ░ ┌─┴─┐  │    │                  ░  ├──────────┤»
q_1: ┤ Ry(θ[1]) ├┤ Rz(θ[5]) ├─░─┤ X ├──┼────┼────■────■────────░──┤ Ry(θ[9]) ├»
     ├──────────┤├──────────┤ ░ └───┘┌─┴─┐  │  ┌─┴─┐  │        ░ ┌┴──────────┤»
a_0: ┤ Ry(θ[2]) ├┤ Rz(θ[6]) ├─░──────┤ X ├──┼──┤ X ├──┼────■───░─┤ Ry(θ[10]) ├»
     ├──────────┤├──────────┤ ░      └───┘┌─┴─┐└───┘┌─┴─┐┌─┴─┐ ░ ├───────────┤»
a_1: ┤ Ry(θ[3]) ├┤ Rz(θ[7]) ├─░───────────┤ X ├─────┤ X ├┤ X ├─░─┤ Ry(θ[11]) ├»
     └──────────┘└──────────┘ ░           └───┘     └───┘└───┘ ░ └───────────┘»
«     ┌───────────┐          
«q_0: ┤ Rz(θ[12]) ├──■───────
«     ├───────────┤  │       
«q_1: ┤ Rz(θ[13]) ├──┼────■──
«     ├───────────┤┌─┴─┐  │  
«a_0: ┤ Rz(θ[14]) ├┤ X ├──┼──
«     ├───────────┤└───┘┌─┴─┐
«a_1: ┤ Rz(θ[15]) ├─────┤ X ├
«     └───────────┘     └───┘

In [5]:
brute_result = brute_force_optimization(hamiltonian,ansatz,x0,beta = beta)

(-0.3862943611198908+0j)
(0.8713544454280923+0j)
(0.029883460620195756+0j)
(0.029883460620195867+0j)
(0.02988346062019598+0j)
(-0.38629436111989046+0j)
(-0.38629436111989046+0j)
(-0.38629436111989046+0j)
(-0.38629436111989046+0j)
(0.029883460620195645+0j)
(0.029883460620195645+0j)
(0.029883460620195867+0j)
(0.02988346062019609+0j)
(-0.38629436111989046+0j)
(-0.3862943611198908+0j)
(-0.3862943611198908+0j)
(-0.3862943611198908+0j)
(-0.604739867170132+0j)
(0.32165802867968796+0j)
(-0.5730677103875761+0j)
(-0.7774279224994772+0j)
(-0.757564617424723+0j)
(-0.7905224072177003+0j)
(-0.7435749543028881+0j)
(-0.7690554283556476+0j)
(-0.7617131304448624+0j)
(-0.7909212771209226+0j)
(-0.7793663947576442+0j)
(-0.7905111605154744+0j)
(-0.6735714060121394+0j)
(-0.7901190906518306+0j)
(-0.7539122202799259+0j)
(-0.7904263961299917+0j)
(-0.7883366106738519+0j)
(-0.7902628015248316+0j)
(-0.7822867818224106+0j)
(-0.7786282264430917+0j)
(-0.7674569571863255+0j)
(-0.7529099311801976+0j)
(-0.76465821619161

In [6]:
state = state_from_ansatz(ansatz,brute_result.x)
from qiskit.quantum_info import entropy
entropy(state)

1.5270653383929498

In [7]:
printarray(state)

[[ 0.06 -0.   -0.   -0.  ]
 [-0.    0.44  0.    0.  ]
 [-0.    0.    0.44 -0.  ]
 [-0.    0.   -0.    0.06]]


In [8]:
x0.shape

(16,)